In [8]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
    import torch
    from IPython.core.display import HTML
    from IPython.display import display
    import os
    print("Running in Google Colab")
    if not torch.cuda.is_available():
        display(HTML("""<div style="background-color: red; font-weight: bold; color: white;">You have not activated a GPU in Google Colab. Follow the instructions in the <code style="color: white;">README</code></div>"""))
    print("Installing requirements")
    requirements_url = "https://raw.githubusercontent.com/willdalh/ml-course/main/requirements.txt"
    if not os.path.exists('requirements.txt'):
        !wget {requirements_url}
    %pip install --user -q -r requirements.txt

## PyTorch

PyTorch er et av flere biblioteker som brukes til trening av dype nevrale nettverk. Det andre kjente alternativet er TensorFlow. Hva man velger avhenger mye av smak, men begge har sine fordeler og ulemper. I noen tilfeller er PyTorch foretrukket fordi det er mer tydelig hvordan treningsprosessen av nevrale nettverk foregår. 

In [11]:
import torch

## Vektorer og matriser - generalisert som tensorer
Fra matematikken vet vi at en liste med feks. 3 tall definerer en vektor/punkt som lever i et tre-dimensjonalt rom. I PyTorch representeres dette gjennom datatypen `Tensor`. 

In [12]:
data = [3, 2, 1] # Normal python list
vector = torch.Tensor(data) # Creating a tensor from it
vector

tensor([3., 2., 1.])

Med tensorer er det flere operasjoner som kan gjøres (som vi også til en grad kan forvente av native Python-lister).

In [13]:
print(f"Max value: {vector.max()}")
print(f"Min value: {vector.min()}")

print(f"Index of max element: {vector.argmax()}")
print(f"Index of min element: {vector.argmin()}")

print(f"Sum: {vector.sum()}")

print(f"Sorted:  {torch.sort(vector).values}")

Max value: 3.0
Min value: 1.0
Index of max element: 0
Index of min element: 2
Sum: 6.0
Sorted:  tensor([1., 2., 3.])


På lik linje har vi et 16-dimensjonalt punkt (som vi heller lager med tilfeldige tall):

In [14]:
high_dim_vector = torch.randint(low=1, high=10, size=(16,))
high_dim_vector

tensor([3, 7, 5, 1, 8, 8, 1, 4, 5, 9, 2, 6, 3, 4, 8, 4])

En matrise er en rektangulær liste / tabell. Det er her `torch.Tensor` begynner å divergere litt fra vanlige Python-lister.  
Vi lager en $2 \times 3$-matrise, altså med 2 rader og 3 kolonner.

In [15]:
matrix = torch.randint(low=1, high=10, size=(2, 3))
matrix

tensor([[3, 1, 2],
        [6, 3, 4]])

Størrelsen og formen på matrisen kaller vi for _shape_. Er vi usikker på hvilken shape matrisen har, kan vi sjekke det med `.shape` attributtet.

In [16]:
print(f"The shape of the matrix is {matrix.shape}")
rows, cols = matrix.shape
print(f"It has {rows} rows and {cols} columns")

The shape of the matrix is torch.Size([2, 3])
It has 2 rows and 3 columns


Vi kan gå videre ved å legge til dybde på en matrise.

In [17]:
tensor = torch.randint(low=0, high=10, size=(2, 3, 4))
print(f"The shape of the tensor is {tensor.shape}")
tensor

The shape of the tensor is torch.Size([2, 3, 4])


tensor([[[6, 9, 8, 4],
         [8, 3, 4, 7],
         [6, 9, 4, 6]],

        [[1, 2, 3, 4],
         [0, 4, 9, 2],
         [3, 0, 2, 0]]])

Tensoren over består av 2 matriser som hver har 3 rader og 4 kolonner.

I praksis brukes ordene _vektor_ og _tensor_ om hverandre. Begge beskriver en samling verdier strukturert med en vilkårlig _shape_. I bunn og grunn er de flerdimensjonale lister/arrays. 

Her kommer det et veiskille i hva man legger i begrepet _dimensjoner_ når man refererer til en tensor. Det kan bety to ting:
- Antall elementer til sammen i tensoren (riktig matematisk sett)
- Antall elementer som ligger i resultatet av `.shape`-attributtet 

For å minske forvirringen kaller man ofte den første for antall _features_. Dette kommer av at datapunkt fra et datasett vil være representert som en vektor, og derfor har den et sett med egenskaper som er en samling numeriske verdier. 

Den andre kan vi kalle for shape-dimensjoner. 

### Funksjonalitet utover det man finner i vanlige Python-lister  
Tensorer er veldig fleksible. Vi kan for eksempel slå sammen radene og kolonnene ved å utføre en `reshape`.

In [18]:
print(f"Original shape: {tensor.shape}")
reshaped = tensor.reshape(2, 3*4) # Combine the rows and columns into one
print(f"Shape of the reshaped tensor: {reshaped.shape}")
reshaped

Original shape: torch.Size([2, 3, 4])
Shape of the reshaped tensor: torch.Size([2, 12])


tensor([[6, 9, 8, 4, 8, 3, 4, 7, 6, 9, 4, 6],
        [1, 2, 3, 4, 0, 4, 9, 2, 3, 0, 2, 0]])

Hvis det er en shape-dimensjon vi ikke har styr på og ofte varierer kan vi la PyTorch regne det ut selv ved å spesifisere `-1` for **en** av posisjonene.

Dette er veldig nyttig når vi ønsker å samle flere datapunkt i en tensor. Vi vet gjerne hvordan et datapunkt ser ut, feks et bilde vil ha en bestemt oppløsning og antall fargekanaler. Disse verdiene vil være felles for alle datapunkt i datasettet. Denne prosessen kalles for batching, og vi kommer tilbake til det senere. 

In [19]:
reshaped = tensor.reshape(-1, 3*4)
reshaped

tensor([[6, 9, 8, 4, 8, 3, 4, 7, 6, 9, 4, 6],
        [1, 2, 3, 4, 0, 4, 9, 2, 3, 0, 2, 0]])

Når vi har flere _shape_-dimensjoner har vi muligheten til å spesifisere dimensjoner av interesse for noen av operasjonene.

In [20]:
tensor = torch.randint(low=0, high=10, size=(2, 3))
tensor

tensor([[4, 1, 5],
        [0, 0, 3]])

In [21]:
print(f"Summing over rows: {tensor.sum(dim=0)}")
print(f"Summing over columns: {tensor.sum(dim=1)}")

Summing over rows: tensor([4, 1, 8])
Summing over columns: tensor([10,  3])


In [22]:
print(f"Min values over rows: {tensor.min(dim=0).values}")
print(f"Min values over columns: {tensor.min(dim=1).values}")

Min values over rows: tensor([0, 0, 3])
Min values over columns: tensor([1, 0])


### Legge til og fjerne tomme dimensjoner
Antall dimensjoner som er i shapen til en tensor kan være forskjellig, mens dataen tilsynelatende har en lik struktur. 
Vi starter med en vektor som inneholder 4 features. Denne legger vi til en _tom_ shape-dimensjon på med `.unsqueeze(dim=X)`

In [23]:
tensor = torch.randint(low=0, high=10, size=(4,))
print(f"Original tensor is {tensor} with shape {tensor.shape}")
modified = tensor.unsqueeze(0)
print(f"Modified tensor is {modified} with shape {modified.shape}")

Original tensor is tensor([6, 4, 1, 8]) with shape torch.Size([4])
Modified tensor is tensor([[6, 4, 1, 8]]) with shape torch.Size([1, 4])


Vi kan også fjerne tomme dimensjoner.

In [24]:
tensor = torch.randint(low=0, high=10, size=(4, 1)) # 4 rows and 1 column
tensor

tensor([[7],
        [3],
        [9],
        [3]])

In [25]:
modified = tensor.squeeze(1)
modified

tensor([7, 3, 9, 3])

PyTorch er litt streng, og vi kommer til å oppleve at den setter krav til hvordan shape tensorene våre skal ha. Nevrale nettverk som vi bygger med PyTorch (mer om det i neste del) forventer at input-dataen kommer som flere datapunkt samlet. Hvis vi bare skal sende inn et datapunkt må vi dytte inn en tom batch-dimensjon framst i tensoren. 

## Flytting av tensorer mellom enheter
Vi skriver PyTorch-kode gjennom Python, men mye av beregningene foregår i et C++ backend. Dette er nødvendig for at ressurskrevende operasjoner kan gjøres på grafikkort. Det eneste vi trenger å gjøre er å flytte tensorer dit. 

In [26]:
print(f"Default tensor location: {tensor.device}")

Default tensor location: cpu


In [27]:
print(f"GPU is available: {torch.cuda.is_available()}")

GPU is available: False


In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

[CUDA](https://en.wikipedia.org/wiki/CUDA) er API-et som brukes for å kjøre beregninger på NVIDIA-grafikkort. 

In [29]:
tensor = tensor.to(device)
print(f"Tensor location: {tensor.device}")

Tensor location: cpu
